In [1]:
import numpy as np

def mesafe_hesapla(x, xi):
    """
    İki vektör (x ve xi) arasındaki Öklid mesafesini hesapla.
    """
    return np.sqrt(np.sum((x - xi) ** 2))

def fknn(x, W, u_labels, K, m=2):
    """
    Bulanık k-En Yakın Komşular (FKNN) algoritması.

    Parametreler:
    - x: Sınıfı bilinmeyen örnek vektörü
    - W: Etiketli örneklerin kümesi (numpy array, şekil: (örnek_sayısı, özellik_sayısı))
    - u_labels: W'deki örneklerin sınıflara olan üyelik dereceleri (numpy array, şekil: (örnek_sayısı, sınıf_sayısı))
    - K: En yakın komşu sayısı
    - m: Bulanıklık parametresi (genelde 2 olarak seçilir)

    Döndürür:
    - üyelikler: x'in her sınıfa olan bulanık üyelik dereceleri
    """
    n_samples = W.shape[0]
    mesafeler = []

    # 1. Adım: x ile W'deki tüm örnekler arasındaki mesafeleri hesapla
    for xi in W:
        mesafeler.append(mesafe_hesapla(x, xi))
    
    # 2. Adım: Mesafeleri sırala ve K en yakın komşuyu seç
    en_yakin_indisler = np.argsort(mesafeler)[:K]
    en_yakin_ornekler = W[en_yakin_indisler]
    en_yakin_uyelikler = u_labels[en_yakin_indisler]

    # 3. Adım: x'in her sınıfa olan bulanık üyelik derecesini hesapla
    sinif_sayisi = u_labels.shape[1]
    üyelikler = np.zeros(sinif_sayisi)  # Her sınıf için üyelik hesaplanacak

    for i in range(sinif_sayisi):  # Her sınıf için hesaplama
        pay = 0
        payda = 0
        for j, idx in enumerate(en_yakin_indisler):
            mesafe = mesafeler[idx]
            agirlik = 1 / (mesafe ** (2 / (m - 1)) + 1e-10)  # Bölme hatalarını önlemek için epsilon eklenmiştir
            pay += agirlik * en_yakin_uyelikler[j, i]
            payda += agirlik
        üyelikler[i] = pay / payda

    return üyelikler




# Bulanık k-En Yakın Komşular (FKNN) Algoritması

Bu doküman, FKNN algoritmasının adımlarını, matematiksel temelini ve ilgili Python kodlarını detaylı bir şekilde açıklar. Algoritma, bir örneğin sınıflara olan bulanık üyelik derecelerini hesaplar.

---

## **1. Mesafe Hesaplama**

Bir örnek \\( x \\) ile veri kümesindeki bir örnek \\( x_i \\) arasındaki mesafe şu şekilde hesaplanır:

\\[
d(x, x_i) = \sqrt{\sum_{j=1}^{n} (x_j - x_{i,j})^2}
\\]

Bu, klasik Öklid mesafesidir ve \\( x \\) ile \\( x_i \\) arasındaki benzerliği ölçer. Python kodunda bu işlem, iki vektör arasındaki kare farkların toplamının karekökünü alarak yapılır.

### Kod
```python
def mesafe_hesapla(x, xi):
    return np.sqrt(np.sum((x - xi) ** 2))
```

___
## **2. En Yakın Komşuların Seçimi**

Mesafeler hesaplandıktan sonra, \\( x \\) ile en küçük \\( K \\) mesafeye sahip komşular seçilir. Bu işlem, mesafelerin sıralanması ve ilk \\( K \\) komşunun alınması ile gerçekleştirilir.

### Matematiksel Açıklama

Komşular şu şekilde tanımlanır:

\\[
\text{Komşular: } \{x_{(1)}, x_{(2)}, \ldots, x_{(K)}\}
\\]

Burada:
- \\( x_{(i)} \\): \\( x \\)'e en yakın \\( i \\)-inci komşudur.
- \\( K \\): Seçilecek komşu sayısı.

### Kod

Bu adım, mesafelerin hesaplanması ve \\( K \\) en yakın komşunun seçilmesiyle gerçekleştirilir:

```python
mesafeler = []
for xi in W:
    mesafeler.append(mesafe_hesapla(x, xi))

# Mesafeleri sırala ve K en yakın komşuyu seç
en_yakin_indisler = np.argsort(mesafeler)[:K]

# Seçilen komşuların üyelik derecelerini al
en_yakin_uyelikler = u_labels[en_yakin_indisler]
```
___

## **3. Bulanık Üyelik Derecelerinin Hesaplanması**

Bir örneğin \\( i \\)-inci sınıfa olan bulanık üyelik derecesi, komşuların mesafelerine ve sınıf üyelik derecelerine dayalı olarak hesaplanır. Yakın olan komşuların katkısı daha yüksek, uzak olanların katkısı daha düşük olacaktır.

### Matematiksel Açıklama

Bir sınıfa olan bulanık üyelik derecesi şu formülle hesaplanır:

\\[
u_i(x) = \frac{\sum_{j=1}^{K} u_{ij} \cdot \left( \frac{1}{\|x - x_j\|^{\frac{2}{m-1}}} \right)}{\sum_{j=1}^{K} \left( \frac{1}{\|x - x_j\|^{\frac{2}{m-1}}} \right)}
\\]

Burada:
- \\( u_i(x) \\): \\( x \\)'in \\( i \\)-inci sınıfa olan bulanık üyelik derecesi.
- \\( u_{ij} \\): \\( j \\)-inci komşunun \\( i \\)-inci sınıfa olan üyelik derecesi.
- \\( \|x - x_j\| \\): \\( x \\) ile \\( j \\)-inci komşu arasındaki mesafe.
- \\( m \\): Bulanıklık parametresi (genellikle \\( m = 2 \\)).

Bu formülde:
- **Pay**: Komşuların sınıf üyelik derecelerinin mesafeye bağlı ağırlıklarla çarpılıp toplanmasıdır:
  \\[
  \text{Pay} = \sum_{j=1}^{K} u_{ij} \cdot w_j
  \\]
  Burada \\( w_j \\), mesafeye dayalı ağırlığı ifade eder:
  \\[
  w_j = \frac{1}{\|x - x_j\|^{\frac{2}{m-1}}}
  \\]

- **Payda**: Komşuların mesafeye dayalı ağırlıklarının toplamıdır:
  \\[
  \text{Payda} = \sum_{j=1}^{K} w_j
  \\]

Sonuç olarak, \\( i \\)-inci sınıfa olan üyelik derecesi:
\\[
u_i(x) = \frac{\text{Pay}}{\text{Payda}}
\\]


### Kod ve Açıklama

Bu işlem, her sınıf için şu şekilde kodlanır:

#### **1. Mesafelerin ve Ağırlıkların Hesaplanması**
Komşuların mesafeleri ve bu mesafelere dayalı ağırlıkları hesaplanır. Ağırlıklar, mesafelerin tersine orantılıdır:

```python
mesafe = mesafeler[idx]  # İlgili komşunun mesafesi
agirlik = 1 / (mesafe ** (2 / (m - 1)) + 1e-10)  # Mesafeye dayalı ağırlık
```

Epsilon değeri, sıfıra bölme hatalarını önlemek için kullanılır.


### **Epsilon Değeri**

Epsilon değeri (\\( 1e-10 \\)), sıfıra bölme hatalarını önlemek için kullanılır. Eğer mesafe \\( \|x - x_j\| \\) sıfıra çok yakın bir değer alırsa, \\( w_j = \frac{1}{\|x - x_j\|^{\frac{2}{m-1}}} \\) ifadesi aşırı büyük değerlere yol açabilir. Epsilon bu durumda, bölme işleminin sayısal kararlılığını garanti altına alır.

---

### **Pay ve Payda Hesaplaması**

**Pay**: \\( K \\) en yakın komşunun mesafelerine ve sınıf üyelik derecelerine dayalı olarak hesaplanır. Her komşunun \\( i \\)-inci sınıfa olan üyelik derecesi, mesafeye dayalı ağırlığı ile çarpılır ve bu değerlerin toplamı alınır:

\\[
\text{Pay} = \sum_{j=1}^{K} u_{ij} \cdot w_j
\\]

Kodda bu işlem şu şekilde yapılır:
```python
pay += agirlik * en_yakin_uyelikler[j, i]
```


### **Payda Hesaplaması**

**Payda**: Komşuların sadece mesafeye dayalı ağırlıklarının toplamıdır. Bu, tüm komşuların \\( x \\)'e olan mesafelerini kullanarak, mesafeye ters orantılı bir şekilde hesaplanır:

\\[
\text{Payda} = \sum_{j=1}^{K} w_j
\\]

Burada \\( w_j \\), \\( j \\)-inci komşunun ağırlığıdır ve şu şekilde hesaplanır:

\\[
w_j = \frac{1}{\|x - x_j\|^{\frac{2}{m-1}}}
\\]

Kodda bu işlem, \\( K \\) komşunun her biri için tekrarlanarak gerçekleştirilir:

```python
payda += agirlik
```

### **Üyelik Derecesinin Hesaplanması**

Her sınıf için bulanık üyelik derecesi, payın paydaya bölünmesiyle elde edilir. Bu hesaplama, \\( x \\)'in ilgili sınıfa olan aitlik derecesini belirler:

\\[
u_i(x) = \frac{\text{Pay}}{\text{Payda}}
\\]

Burada:
- **Pay**: Komşuların sınıf üyelik derecelerinin, mesafelere dayalı ağırlıklarıyla çarpılarak toplamıdır.
- **Payda**: Sadece mesafelere dayalı ağırlıkların toplamıdır.

Kodda bu işlem şu şekilde gerçekleştirilir:

```python
üyelikler[i] = pay / payda
```
Bu işlem, her sınıf için ayrı ayrı tekrarlanır ve tüm sınıflar için \\( u_i(x) \\) değerleri hesaplanır.
___
### **Tam Kod**

Aşağıda, \\( x \\)'in her sınıfa olan üyelik derecelerini hesaplayan tam kod bulunmaktadır. Bu kod, mesafelerin hesaplanması, en yakın \\( K \\) komşunun seçilmesi ve her sınıf için üyelik derecelerinin hesaplanmasını içerir:

```python
def fknn(x, W, u_labels, K, m=2):
    mesafeler = []
    for xi in W:
        mesafeler.append(mesafe_hesapla(x, xi))
    
    # En yakın K komşuyu seç
    en_yakin_indisler = np.argsort(mesafeler)[:K]
    en_yakin_uyelikler = u_labels[en_yakin_indisler]

    # Sınıf sayısını belirle
    sinif_sayisi = u_labels.shape[1]
    üyelikler = np.zeros(sinif_sayisi)

    # Her sınıf için üyelik derecelerini hesapla
    for i in range(sinif_sayisi):
        pay = 0
        payda = 0
        for j, idx in enumerate(en_yakin_indisler):
            mesafe = mesafeler[idx]
            agirlik = 1 / (mesafe ** (2 / (m - 1)) + 1e-10)  # Mesafeye dayalı ağırlık
            pay += agirlik * en_yakin_uyelikler[j, i]  # Payı güncelle
            payda += agirlik  # Paydayı güncelle
        üyelikler[i] = pay / payda  # Üyelik derecesini hesapla

    return üyelikler
```
___
### **Kodun İşleyişi**

Bu kod, \\( x \\)'in her sınıfa olan bulanık üyelik derecelerini hesaplamak için aşağıdaki adımları izler:

1. **Mesafelerin Hesaplanması**: 
   \\( x \\)'in, veri kümesindeki her bir örnekle olan mesafeleri hesaplanır. Mesafeler, Öklid metrik kullanılarak hesaplanır:

 ```python
mesafeler = []
for xi in W:
    mesafeler.append(mesafe_hesapla(x, xi))
```
2. **En Yakın Komşuların Seçilmesi**:
   Hesaplanan mesafeler sıralanır ve en küçük \\( K \\) mesafeye sahip olan komşuların indeksleri belirlenir. Bu komşular, \\( x \\)'in sınıf üyelik derecelerinin belirlenmesinde kullanılır:

```python
        en_yakin_indisler = np.argsort(mesafeler)[:K]
        en_yakin_uyelikler = u_labels[en_yakin_indisler]
```

3. **Her Sınıf İçin Pay ve Paydanın Hesaplanması.**: Her sınıf için:
- **Pay**: Komşuların mesafeye bağlı ağırlıkları ile sınıf üyelik derecelerinin çarpımları toplanır:
```python
pay += agirlik * en_yakin_uyelikler[j, i]
```

- **Payda**: Komşuların yalnızca mesafeye bağlı ağırlıklarının toplamı alınır:
```python
payda += agirlik
```

4. **Üyelik Derecelerinin Hesaplanması**:
   Payın paydaya bölünmesiyle \\( x \\)'in her sınıfa olan bulanık üyelik dereceleri hesaplanır:
```python
üyelikler[i] = pay / payda
```

5. **Sonuçların Döndürülmesi**:
   Tüm sınıflar için hesaplanan üyelik dereceleri bir vektör olarak döndürülür:
```python
return üyelikler
```

In [3]:
# Örnek kullanım
# W: 3 örnekli ve 2 özellikli örnek kümesi
W = np.array([[1, 2], [2, 3], [3, 4]])

# u_labels: Her örneğin 2 sınıfa olan üyelik dereceleri
u_labels = np.array([[0.8, 0.2], [0.4, 0.6], [0.3, 0.7]])

# x: Sınıfı bilinmeyen örnek
x = np.array([2.5, 3.5])

# K: Komşu sayısı
K = 2

# Bulanıklık parametresi
m = 2

# Bulanık üyelik derecelerini hesapla
sonuc = fknn(x, W, u_labels, K, m)
print("x'in her sınıfa olan üyelik dereceleri:", sonuc)

x'in her sınıfa olan üyelik dereceleri: [0.35 0.65]


## **Örnek Kullanım **

FKNN algoritmasının çalışma mantığını daha iyi anlamak için aşağıdaki örneği adım adım ele alacağız.

---

### **1. Verilerin Tanımlanması**

#### **Veri Kümesi (\\( W \\))**:
- Üç örnekten oluşan bir veri kümesi vardır:
  \\[
  W = \begin{bmatrix} 1 & 2 \\ 2 & 3 \\ 3 & 4 \end{bmatrix}
  \\]
  Burada her satır bir örneği, sütunlar ise örneklerin özelliklerini temsil eder.

#### **Sınıf Üyelik Dereceleri (\\( u\_labels \\))**:
- Veri kümesindeki her örnek için sınıflara olan üyelik dereceleri şu şekildedir:
  \\[
  u_{labels} = \begin{bmatrix} 0.8 & 0.2 \\ 0.4 & 0.6 \\ 0.3 & 0.7 \end{bmatrix}
  \\]
  Örneğin:
  - İlk örnek (\\( [1, 2] \\)) için 1. sınıfa olan üyelik derecesi \\( 0.8 \\), 2. sınıfa olan üyelik derecesi \\( 0.2 \\)'dir.
  - İkinci örnek (\\( [2, 3] \\)) için üyelik dereceleri \\( 0.4 \\) ve \\( 0.6 \\)'dır.

#### **Sınıfı Bilinmeyen Örnek (\\( x \\))**:
- Sınıfı bilinmeyen örnek:
  \\[
  x = \begin{bmatrix} 2.5 & 3.5 \end{bmatrix}
  \\]

#### **Diğer Parametreler**:
- En yakın komşu sayısı (\\( K \\)): \\( 2 \\)
- Bulanıklık parametresi (\\( m \\)): \\( 2 \\)

---

### **2. Algoritmanın Çalıştırılması**

FKNN algoritmasının adımlarını takip ederek sınıfı bilinmeyen örneğin sınıflara olan üyelik derecelerini hesaplayacağız.

#### **Adım 1: Mesafelerin Hesaplanması**
Her bir veri kümesi örneği ile \\( x \\) arasındaki Öklid mesafesi hesaplanır:

\\[
d(x, x_1) = \sqrt{(2.5 - 1)^2 + (3.5 - 2)^2} = \sqrt{1.5^2 + 1.5^2} = \sqrt{4.5} \approx 2.12
\\]
\\[
d(x, x_2) = \sqrt{(2.5 - 2)^2 + (3.5 - 3)^2} = \sqrt{0.5^2 + 0.5^2} = \sqrt{0.5} \approx 0.71
\\]
\\[
d(x, x_3) = \sqrt{(2.5 - 3)^2 + (3.5 - 4)^2} = \sqrt{0.5^2 + 0.5^2} = \sqrt{0.5} \approx 0.71
\\]

Sonuç olarak:
\\[
\text{Mesafeler} = \begin{bmatrix} 2.12 & 0.71 & 0.71 \end{bmatrix}
\\]

#### **Adım 2: En Yakın Komşuların Seçilmesi**
Mesafeler sıralanır ve en küçük \\( K = 2 \\) mesafeye sahip komşular seçilir:
- En yakın 2 komşu: \\( x_2 = [2, 3] \\) ve \\( x_3 = [3, 4] \\).

#### **Adım 3: Üyelik Derecelerinin Hesaplanması**

Sınıfı bilinmeyen örnek (\\( x \\)) için her sınıfa olan üyelik dereceleri hesaplanır. Aşağıdaki formül kullanılır:

\\[
u_i(x) = \frac{\sum_{j=1}^{K} u_{ij} \cdot \left( \frac{1}{\|x - x_j\|^{\frac{2}{m-1}}} \right)}{\sum_{j=1}^{K} \left( \frac{1}{\|x - x_j\|^{\frac{2}{m-1}}} \right)}
\\]

**1. Sınıf için Üyelik Derecesi (\\( u_1(x) \\)):**
- Ağırlıklar:
  \\[
  w_2 = \frac{1}{d(x, x_2)^{2}} = \frac{1}{0.71^2} \approx 1.98
  \\]
  \\[
  w_3 = \frac{1}{d(x, x_3)^{2}} = \frac{1}{0.71^2} \approx 1.98
  \\]

- Pay:
  \\[
  \text{Pay} = (w_2 \cdot u_{12}) + (w_3 \cdot u_{13}) = (1.98 \cdot 0.4) + (1.98 \cdot 0.3) = 0.792 + 0.594 = 1.386
  \\]

- Payda:
  \\[
  \text{Payda} = w_2 + w_3 = 1.98 + 1.98 = 3.96
  \\]

- Üyelik:
  \\[
  u_1(x) = \frac{\text{Pay}}{\text{Payda}} = \frac{1.386}{3.96} \approx 0.35
  \\]

**2. Sınıf için Üyelik Derecesi (\\( u_2(x) \\)):**
- Pay:
  \\[
  \text{Pay} = (w_2 \cdot u_{22}) + (w_3 \cdot u_{23}) = (1.98 \cdot 0.6) + (1.98 \cdot 0.7) = 1.188 + 1.386 = 2.574
  \\]

- Payda (önceden hesaplandı): \\( 3.96 \\)

- Üyelik:
  \\[
  u_2(x) = \frac{\text{Pay}}{\text{Payda}} = \frac{2.574}{3.96} \approx 0.65
  \\]

---

### **3. Sonuçlar**

Sınıfı bilinmeyen örnek (\\( x \\)) için sınıf üyelik dereceleri şu şekilde hesaplanmıştır:
\\[
u(x) = \begin{bmatrix} 0.35 & 0.65 \end{bmatrix}
\\]

Bu sonuç, \\( x \\)'in 2. sınıfa ait olma olasılığının daha yüksek olduğunu göstermektedir.

---

### **Kod Çıktısı**

Aşağıdaki kod çalıştırıldığında elde edilen çıktı:
```python
x'in her sınıfa olan üyelik dereceleri: [0.35 0.65]
```


---

### **4. Çıktının Yorumu**

Sonuçlara göre:
- \\( x \\)'in 1. sınıfa ait olma üyelik derecesi \( 0.35 \)'tir.
- \\( x \\)'in 2. sınıfa ait olma üyelik derecesi \( 0.65 \)'tir.

Bu değerler, \\( x \\)'in 2. sınıfa daha yakın olduğunu ve yüksek bir olasılıkla bu sınıfa ait olduğunu gösterir. FKNN algoritması, sadece bir sınıfa kesin bir şekilde atama yapmak yerine, örneğin tüm sınıflara olan üyelik derecelerini döndürerek belirsizliğin daha esnek bir şekilde ifade edilmesini sağlar. Bu, özellikle sınıflar arasında net sınırların olmadığı durumlarda avantajlıdır.

---

### **FKNN'nin Güçlü Yönleri**

1. **Bulanık Üyelik Dereceleri**:
   - FKNN algoritması, örneğin yalnızca bir sınıfa ait olup olmadığını söylemekle kalmaz, aynı zamanda diğer sınıflarla olan üyelik derecelerini de hesaplar. Bu, sınıflar arasında geçişkenliğin olduğu durumlarda daha anlamlı sonuçlar verir.

2. **Belirsizliği Yönetme**:
   - Klasik k-NN yönteminden farklı olarak FKNN, belirsizliği yönetmek için bulanıklık parametresini (\( m \)) kullanır. Bu parametre, sınıflar arasındaki karar mekanizmasını esnek hale getirir.

3. **Ölçeklenebilirlik**:
   - FKNN, \\( K \\) komşuya odaklandığı için büyük veri kümelerinde de etkili bir şekilde çalışabilir.

---

### **FKNN'nin Zayıf Yönleri**

1. **Karmaşıklık**:
   - Algoritma, mesafelerin hesaplanması ve üyelik derecelerinin belirlenmesi için ek hesaplama gerektirir. Bu, özellikle büyük veri kümelerinde performans sorunlarına yol açabilir.

2. **Hiperparametre Seçimi**:
   - \\( K \\) (komşu sayısı) ve \\( m \\) (bulanıklık parametresi) gibi hiperparametrelerin doğru seçimi, algoritmanın performansı için kritiktir.

3. **Mesafe Metriklerine Bağımlılık**:
   - Algoritma, mesafe hesaplamalarına dayandığından, yanlış seçilen mesafe metrikleri (ör. Öklid mesafesi) sonuçları olumsuz etkileyebilir.

---

### **FKNN'nin Uygulama Alanları**

1. **Desen Tanıma**:
   - Görüntü sınıflandırma, yüz tanıma, el yazısı tanıma gibi alanlarda kullanılabilir.

2. **Tıbbi Tanı**:
   - Bir hastanın belirtilerine göre hastalıkların sınıflandırılması gibi durumlarda faydalıdır.

3. **Müşteri Segmentasyonu**:
   - E-ticaret verilerinde müşteri davranışlarının segmentlere ayrılması için kullanılabilir.

4. **Doğal Dil İşleme**:
   - Metin verilerinin sınıflandırılması gibi NLP uygulamalarında kullanılabilir.

---

### **Sonuç**

Bu örnek üzerinden FKNN algoritmasının nasıl çalıştığını ve sonuçların nasıl yorumlandığını gösterdik. FKNN, özellikle sınıflar arasındaki belirsizliğin yüksek olduğu ve geçişkenliğin önemli olduğu durumlarda klasik k-NN yöntemine göre daha avantajlıdır. Ayrıca, bulanık mantık prensiplerini kullanarak hem esneklik hem de hassasiyet sağlar.
